In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import sympy as sp
import torch
import linecache
from pathlib import Path


In [2]:
model_path = "./ckpoints/checkpoint-85000"
tokenizer_path="./models/t5-small"

tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
print(f"max length: {model.config.n_positions}")

device='cuda'
model=model.to(device)
model=model.eval()
def generate_summary(input_tokens):
    inputs = tokenizer(input_tokens, return_tensors="pt",is_split_into_words=True, padding=True, truncation=True)
    if device =='cuda':
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
        outputs = model.generate(inputs['input_ids'], max_length=16, num_beams=2, early_stopping=True)
    else:
        outputs = model.generate(inputs.input_ids, max_length=1024, num_beams=2, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


max length: 512


In [3]:
n=770
test_input=torch.load("./datasets/test/Z_uniform_radius.pt").tolist()[n]
test_input=[str(num) for num in test_input]

print('origin expression:\n{}\n'.format(test_input))



origin expression:
['2287', '20000', '5351', '100000']



/tmp/ipykernel_25227/1557179230.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_input=torch.load("./datasets/test/Z_uniform_radius.pt").tolist()[n]


In [4]:
after_train_str=generate_summary(test_input)
after_train_prefix=after_train_str.split(" ")
print('after train:\n{}\n'.format(after_train_prefix))


after train:
['-7', '1', '-1', '0']



In [5]:
n=770
true_input=torch.load("./datasets/test/A_uniform_radius.pt").tolist()[n]
true_input=[str(num) for num in true_input]
print(true_input)

['-7', '1', '-1', '0']


/tmp/ipykernel_25227/4210257590.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  true_input=torch.load("./datasets/test/A_uniform_radius.pt").tolist()[n]


In [6]:
inp=torch.load("./datasets/test/Z_uniform_radius.pt").tolist()
outp=torch.load("./datasets/test/A_uniform_radius.pt").tolist()

count=0

for i in range(1000):
    if generate_summary([str(num) for num in inp[i]]).split(" ")==[str(num) for num in outp[i]]:
        count+=1
    if i%10==0:
        print(i)

print(count/1000)


/tmp/ipykernel_17740/490705215.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  inp=torch.load("./datasets/test/Z_uniform_radius.pt").tolist()
/tmp/ipykernel_17740/490705

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
0.926


In [6]:
import torch

test_datasets = [
    ("Z_uniform_radius.pt", "A_uniform_radius.pt", "均匀半径"),
    ("Z_hyperbolic_R10.pt", "A_hyperbolic_R10.pt", "双曲 R=10"),
    ("Z_hyperbolic_R2.pt", "A_hyperbolic_R2.pt", "双曲 R=2"),
    ("Z_euclidean.pt", "A_euclidean.pt", "欧几里得"),
    ("Z_midchord.pt", "A_midchord.pt", "弦中点"),

]

# 存储每个数据集的准确率
accuracies = []

print("="*60)
print("开始测试多个数据集")
print("="*60)

# 循环测试每个数据集
for idx, (inp_file, outp_file, dataset_name) in enumerate(test_datasets, 1):
    print(f"\n[{idx}/4] 测试数据集: {dataset_name}")
    print("-"*60)
    
    # 加载数据
    inp = torch.load(f"./datasets/test/{inp_file}").tolist()
    outp = torch.load(f"./datasets/test/{outp_file}").tolist()
    
    # 确定测试样本数量（取前10000个或全部）
    test_size = min(10000, len(inp))
    print(f"测试样本数: {test_size}")
    
    # 计数正确预测
    count = 0
    for i in range(test_size):
        prediction = generate_summary([str(num) for num in inp[i]]).split(" ")
        ground_truth = [str(num) for num in outp[i]]
        
        if prediction == ground_truth:
            count += 1
        
        # 每100个样本打印一次进度
        if i % 100 == 0 and i > 0:
            current_acc = count / (i + 1)
            print(f"  进度: {i}/{test_size} | 当前准确率: {current_acc:.4f}")
    
    # 计算该数据集的准确率
    accuracy = count / test_size
    accuracies.append(accuracy)
    
    print(f"\n✓ {dataset_name} 准确率: {accuracy:.4f} ({count}/{test_size})")

# 计算平均准确率
avg_accuracy = sum(accuracies) / len(accuracies)

# 输出总结
print("\n" + "="*60)
print("测试结果汇总")
print("="*60)

for idx, (_, _, dataset_name) in enumerate(test_datasets):
    print(f"{dataset_name:15} : {accuracies[idx]:.4f} ({accuracies[idx]*100:.2f}%)")

print("-"*60)
print(f"{'平均准确率':15} : {avg_accuracy:.4f} ({avg_accuracy*100:.2f}%)")
print("="*60)

开始测试多个数据集

[1/4] 测试数据集: 均匀半径
------------------------------------------------------------
测试样本数: 10000


/tmp/ipykernel_25227/1109908843.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  inp = torch.load(f"./datasets/test/{inp_file}").tolist()
/tmp/ipykernel_25227/1109908843

  进度: 100/10000 | 当前准确率: 0.9703
  进度: 200/10000 | 当前准确率: 0.9652
  进度: 300/10000 | 当前准确率: 0.9601
  进度: 400/10000 | 当前准确率: 0.9601
  进度: 500/10000 | 当前准确率: 0.9601
  进度: 600/10000 | 当前准确率: 0.9634
  进度: 700/10000 | 当前准确率: 0.9643
  进度: 800/10000 | 当前准确率: 0.9576
  进度: 900/10000 | 当前准确率: 0.9578
  进度: 1000/10000 | 当前准确率: 0.9570
  进度: 1100/10000 | 当前准确率: 0.9573
  进度: 1200/10000 | 当前准确率: 0.9567
  进度: 1300/10000 | 当前准确率: 0.9577
  进度: 1400/10000 | 当前准确率: 0.9586
  进度: 1500/10000 | 当前准确率: 0.9587
  进度: 1600/10000 | 当前准确率: 0.9588
  进度: 1700/10000 | 当前准确率: 0.9588
  进度: 1800/10000 | 当前准确率: 0.9584
  进度: 1900/10000 | 当前准确率: 0.9574
  进度: 2000/10000 | 当前准确率: 0.9570
  进度: 2100/10000 | 当前准确率: 0.9548
  进度: 2200/10000 | 当前准确率: 0.9541
  进度: 2300/10000 | 当前准确率: 0.9526
  进度: 2400/10000 | 当前准确率: 0.9538
  进度: 2500/10000 | 当前准确率: 0.9536
  进度: 2600/10000 | 当前准确率: 0.9546
  进度: 2700/10000 | 当前准确率: 0.9526
  进度: 2800/10000 | 当前准确率: 0.9529
  进度: 2900/10000 | 当前准确率: 0.9524
  进度: 3000/10000 | 当前准确率: 0.9513
  进度: 3100/10000 | 